# JSONReader Ingestion RAG

In [1]:
!pip install llama-index llama-index-embeddings-huggingface llama_index-llms-ollama
!pip install llama-index-readers-json

In [21]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.
import nest_asyncio

nest_asyncio.apply()

In [5]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core import Settings
from llama_index.core.agent import ReActAgent
from llama_index.core.indices.struct_store import JSONQueryEngine
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama

In [15]:
# Set up OpenAI
import os
import getpass
import openai

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
openai.api_key = os.environ["OPENAI_API_KEY"]

OpenAI API Key:········


# RAG Search on An Academic Paper in Llama-Index

In [9]:
# Set LLM
embed_model = HuggingFaceEmbedding(model_name='sentence-transformers/msmarco-distilbert-base-v4')
llm = Ollama(model="llama3", temperature=0)

## JSON Reader

https://llamahub.ai/l/readers/llama-index-readers-json

In [3]:
from llama_index.readers.json import JSONReader

# Initialize JSONReader
reader = JSONReader()

background_json = "./data/pubmed_background.json"

# Load data from JSON file
background_docs = reader.load_data(input_file=background_json, extra_info={})

In [12]:
storage_dir = "./data"

# Load documents into index
index = VectorStoreIndex.from_documents(background_docs, embed_model=embed_model)
index.storage_context.persist(persist_dir=storage_dir)
print('Created vector DB')

Created vector DB


In [15]:
# configure retriever
query_engine = index.as_query_engine(mode="context", llm=llm, embed_model=embed_model, similarity_top_k=10, verbose=True)

In [16]:
question = "what's RAG in the paper?"
response = query_engine.query(question)
print(response)

ReadTimeout: timed out

In [ ]:
print(response.source_nodes[0].text)

In [ ]:
question = "who are the authors of the Retrieval-Augmented Generation (aka RAG) paper? The answer should be on the first page."
response = query_engine.query(question)
print(response)

In [ ]:
response